In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

model = keras.models.load_model('stock_prediction_model.keras')

In [2]:
from datetime import datetime, timedelta

now = datetime.now()
time_delta = timedelta(days=150)
start = now - time_delta
end = now

print(f"From: {start} to {end}")

From: 2025-05-02 18:03:40.723113 to 2025-09-29 18:03:40.723113


In [3]:
ticker = "^NSEI"
df = yf.download(ticker, start, end)
df.head

C:\Users\Krish Patel\AppData\Local\Temp\ipykernel_7640\410058184.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start, end)
[*********************100%***********************]  1 of 1 completed


<bound method NDFrame.head of Price              Close          High           Low          Open  Volume
Ticker             ^NSEI         ^NSEI         ^NSEI         ^NSEI   ^NSEI
Date                                                                      
2025-05-02  24346.699219  24589.150391  24238.500000  24311.900391  421100
2025-05-05  24461.150391  24526.400391  24400.650391  24419.500000  291500
2025-05-06  24379.599609  24509.650391  24331.800781  24500.750000  302400
2025-05-07  24414.400391  24449.599609  24220.000000  24233.300781  330100
2025-05-08  24273.800781  24447.250000  24150.199219  24431.500000  411400
...                  ...           ...           ...           ...     ...
2025-09-23  25169.500000  25261.900391  25084.650391  25209.000000  299200
2025-09-24  25056.900391  25149.849609  25027.449219  25108.750000  244400
2025-09-25  24890.849609  25092.699219  24878.300781  25034.500000  342500
2025-09-26  24654.699219  24868.599609  24629.449219  24818.550781  29

In [4]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Close']])  # Use only the 'Close' column

# Prepare the input for prediction
x_input = np.array([scaled_data[-100:]])  # Last 100 rows
x_input = x_input.reshape(1, 100, 1)  # Reshape to (1, 100, 1)

# Predict today's price
y_predicted = model.predict(x_input)

# Inverse transform the prediction to get the actual price
y_predicted = scaler.inverse_transform(y_predicted)

print(f"Predicted Price for Today: {y_predicted.flatten()[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
Predicted Price for Today: 24646.076171875
